# II - Single GPU Training
In the previous notebok we set up our pool of GPU nodes. In this notebook we are going to get one of the nodes in the pool to train a deep learning model for small number of epochs. The model and results of the training will be then loaded into blob storage for later retrieval.

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Delete job](#section4)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [ ]:
shipyard

Get some variables stored in the Setup notebook:

In [ ]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>
## Configure job
We need to configure our job to download training and test data. The following code simply randomly selects servers to download this data from and is a byproduct of ensuring sufficient bandwidth for attendees downloading concurrently during the workshop. This random selection code and distribution of data would not be needed for typical training jobs. Typically, training data would be stored in your Azure Storage account instead.

In [ ]:
import random

CNTK_TRAIN_DATA_FILE = 'Train_cntk_text.txt'
CNTK_TEST_DATA_FILE = 'Test_cntk_text.txt'
URL_FMT = 'https://{}.blob.core.windows.net/{}/{}'

def select_random_data_storage_container():
    """Randomly select a storage account and container for CNTK train/test data.
    This is specific for the workshop to help distribute attendee load. This
    function will only work on Python2"""
    ss = random.randint(0, 4)
    cs = random.randint(0, 4)
    sa = '{}{}bigai'.format(ss, chr(ord('z') - ss))
    cont = '{}{}{}'.format(cs, chr(ord('i') - cs * 2), chr(ord('j') - cs * 2))
    return sa, cont

def create_resource_file_list():
    sa, cont = select_random_data_storage_container()
    ret = [{
        'file_path': CNTK_TRAIN_DATA_FILE,
        'blob_source': URL_FMT.format(sa, cont, CNTK_TRAIN_DATA_FILE)
    }]
    sa, cont = select_random_data_storage_container()
    ret.append({
        'file_path': CNTK_TEST_DATA_FILE,
        'blob_source': URL_FMT.format(sa, cont, CNTK_TEST_DATA_FILE)
    })
    return ret


In the dictonary below we define the properties of the job we wish to execute. You can see that we have specified that the image to use is the one we defined at the beginning of this notebook. Another interesting note is that we specify the gpu switch to true since we want the job to use the GPU. Finally the command is as follows:

```
source /cntk/activate-cntk
python /code/ConvNet_CIFAR10.py
```

Which in essence activates the CNTK Anaconda environment then runs the **ConvNet_CIFAR10.py** script which will train and evaluate the model.

In the jobs json below, `resource_files` contains the URLs for CNTK training and test data for this script.

In [ ]:
TASK_ID = 'run_cifar10' # This should be changed per task

JOB_ID = 'cntk-training-job'

COMMAND = 'bash -c "source /cntk/activate-cntk; python -u /code/ConvNet_CIFAR10.py"'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": [
                {
                    "id": TASK_ID,
                    "image": IMAGE_NAME,
                    "remove_container_after_exit": True,
                    "command": COMMAND,
                    "gpu": True,
                    "resource_files": create_resource_file_list(),
                    "output_data": {
                        "azure_storage": [
                            {
                                "storage_account_settings": STORAGE_ALIAS,
                                "container": "output",
                                "source": "$AZ_BATCH_TASK_WORKING_DIR/Models"
                            },
                        ]
                    },
                }
            ],
        }
    ]
}

Write the jobs configuration to the `jobs.json` file:

In [ ]:
import json
import os

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs


In [ ]:
shipyard pool listnodes

Now that we have confirmed everything is working we can execute our job using the command below. The tail switch at the end will stream stdout from the node.

In [ ]:
shipyard jobs add --tail stdout.txt

We can also retrieve this `stdout.txt` data independently of `--tail` above by using the `data stream` command. Note that when you delete the job all this information is also deleted.

In [ ]:
shipyard data stream --filespec $JOB_ID,$TASK_ID,stdout.txt

If something goes wrong you can run the following command to get the stderr output from the job.

In [ ]:
shipyard data stream --filespec $JOB_ID,$TASK_ID,stderr.txt

<a id='section4'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [ ]:
shipyard jobs del -y --termtasks --wait

[Next notebook: Scoring](03_Scoring_model.ipynb)